# Libraries

In [1]:
from src import paths
import pandas as pd
import numpy as np

# Dataset


In [2]:
stable_ndvi_path = paths.data_interim_dir("stable", "stable_ndvi.csv")
drought_ndvi_path = paths.data_interim_dir("drought", "drought_ndvi.csv")

In [3]:
stable_df = pd.read_csv(stable_ndvi_path, index_col=["ID", "IDpix"])
drought_df = pd.read_csv(drought_ndvi_path, index_col=["ID", "IDpix"])

In [4]:
non_change_df = pd.concat((stable_df, drought_df), axis=0)

In [5]:
metadata_columns = ["lat", "lon", "change_type", "change_start", "vegetation_type"]
non_change_signal_columns = list(filter(lambda col: col not in metadata_columns, (col for col in non_change_df.columns)))

In [6]:
non_change_metadata_df = non_change_df[metadata_columns]

In [7]:
non_change_signal_df = non_change_df[non_change_signal_columns]
non_change_signal_df.columns = pd.to_datetime(non_change_signal_df.columns)
non_change_signal_df = non_change_signal_df.reindex(sorted(non_change_signal_df.columns), axis=1)

In [8]:
non_change_signal_df

2000-01-03  2000-01-04  2000-01-11  2000-01-18  2000-01-20  \
ID  IDpix                                                               
0   1             NaN         NaN    0.707001         NaN         NaN   
    2             NaN         NaN    0.730423         NaN         NaN   
    3             NaN         NaN    0.746536         NaN         NaN   
    4             NaN         NaN    0.750166         NaN         NaN   
    5             NaN         NaN    0.765939         NaN         NaN   
...               ...         ...         ...         ...         ...   
326 6579          NaN         NaN    0.355533         NaN         NaN   
    6580          NaN         NaN    0.392213         NaN         NaN   
    6581          NaN         NaN    0.428320         NaN         NaN   
    6582          NaN         NaN    0.442681         NaN         NaN   
    6583          NaN         NaN    0.442681         NaN         NaN   

           2000-01-26  2000-01-27  2000-01-28  2000-02-03  2000-02-05  ...  \
ID  IDpix                                                              ...   
0   1             NaN    0.723551         NaN         NaN         NaN  ...   
    2             NaN    0.720345         NaN         NaN         NaN  ...   
    3             NaN    0.741756         NaN         NaN         NaN  ...   
    4             NaN    0.741654         NaN         NaN         NaN  ...   
    5             NaN    0.749571         NaN         NaN         NaN  ...   
...               ...         ...         ...         ...         ...  ...   
326 6579          NaN    0.349749         NaN         NaN         NaN  ...   
    6580          NaN    0.388196         NaN         NaN         NaN  ...   
    6581          NaN    0.416210         NaN         NaN         NaN  ...   
    6582          NaN    0.414121         NaN         NaN         NaN  ...   
    6583          NaN    0.414121         NaN         NaN         NaN  ...   

           2022-12-08  2022-12-09  2022-12-10  2022-12-14  2022-12-16  \
ID  IDpix                                                               
0   1             NaN    0.723368         NaN    0.615958         NaN   
    2             NaN    0.732564         NaN    0.655706         NaN   
    3             NaN    0.764361         NaN    0.695667         NaN   
    4             NaN    0.765921         NaN    0.735234         NaN   
    5             NaN    0.761045         NaN    0.710822         NaN   
...               ...         ...         ...         ...         ...   
326 6579          NaN    0.367841         NaN    0.407246         NaN   
    6580          NaN    0.403706         NaN    0.416060         NaN   
    6581          NaN    0.388647         NaN    0.395280         NaN   
    6582          NaN    0.374593         NaN    0.385955         NaN   
    6583          NaN    0.374593         NaN    0.385955         NaN   

           2022-12-17  2022-12-18  2022-12-19  2022-12-24  2022-12-25  
ID  IDpix                                                              
0   1        0.683760         NaN         NaN         NaN    0.660891  
    2        0.755396         NaN         NaN         NaN    0.734281  
    3        0.774370         NaN         NaN         NaN    0.749191  
    4        0.772314         NaN         NaN         NaN    0.746010  
    5        0.760751         NaN         NaN         NaN    0.744337  
...               ...         ...         ...         ...         ...  
326 6579     0.356350         NaN         NaN         NaN    0.328377  
    6580     0.392987         NaN         NaN         NaN    0.380575  
    6581     0.365728         NaN         NaN         NaN    0.351286  
    6582     0.363775         NaN         NaN         NaN    0.345881  
    6583     0.363775         NaN         NaN         NaN    0.345881  

[8115 rows x 2377 columns]

## Sample 30% of polygons for esn training

In [9]:
num_polygons = non_change_metadata_df.index.get_level_values(0).unique().size
esn_n = int(num_polygons * 0.3)
fault_detection_n = num_polygons - esn_n

In [10]:
esn_n, fault_detection_n

(44, 105)

In [11]:
rng = np.random.default_rng(0)

In [12]:
esn_indices = pd.Index(rng.choice(non_change_metadata_df.index.get_level_values(0).unique(), size=esn_n, replace=False))
fault_detection_indices = non_change_metadata_df.index.get_level_values(0).unique()[~non_change_metadata_df.index.get_level_values(0).unique().isin(esn_indices)]

assert (non_change_metadata_df.index.get_level_values(0).unique() == pd.Index.union(esn_indices, fault_detection_indices)).all()

In [ ]:
non_change_fault_detection_metadata_df = non_change_metadata_df.loc[fault_detection_indices]
non_change_fault_detection_signal_df = non_change_signal_df.loc[fault_detection_indices]

In [13]:
esn_metadata_df = non_change_metadata_df.loc[esn_indices]
esn_signal_df = non_change_signal_df.loc[esn_indices]

## Fault detection dataset

In [14]:
logging_ndvi_path = paths.data_interim_dir("logging", "logging_ndvi.csv")
fire_ndvi_path = paths.data_interim_dir("fire", "fire_ndvi.csv")

logging_df = pd.read_csv(logging_ndvi_path, index_col=["ID", "IDpix"])
fire_df = pd.read_csv(fire_ndvi_path, index_col=["ID", "IDpix"])

In [15]:
change_df = pd.concat((logging_df, fire_df), axis=0)

In [16]:
change_signal_columns = list(filter(lambda col: col not in metadata_columns, (col for col in change_df.columns)))

In [17]:
change_metadata_df = change_df[metadata_columns]

In [18]:
change_signal_df = change_df[change_signal_columns]
change_signal_df.columns = pd.to_datetime(change_signal_df.columns)
change_signal_df = change_signal_df.reindex(sorted(change_signal_df.columns), axis=1)

### Create dummy event dates for non-change polygons

In [19]:
non_change_event_dates = pd.Series(non_change_fault_detection_signal_df.columns.max() +  pd.DateOffset(years=1), index=non_change_fault_detection_signal_df.index) # dummy event date for non_change signal

In [20]:
non_change_fault_detection_metadata_df["change_start"] = non_change_event_dates

In [21]:
non_change_fault_detection_metadata_df

lat        lon change_type change_start vegetation_type
ID  IDpix                                                               
2   87    -33.058766 -71.004412      stable   2023-12-25          native
    88    -33.059036 -71.004951      stable   2023-12-25          native
    89    -33.058227 -71.003334      stable   2023-12-25          native
    90    -33.058497 -71.003065      stable   2023-12-25          native
    91    -33.058497 -71.003873      stable   2023-12-25          native
...              ...        ...         ...          ...             ...
326 6579  -33.647073 -71.043219     drought   2023-12-25          native
    6580  -33.647881 -71.044836     drought   2023-12-25          native
    6581  -33.647342 -71.043758     drought   2023-12-25          native
    6582  -33.648420 -71.044836     drought   2023-12-25          native
    6583  -33.647612 -71.044297     drought   2023-12-25          native

[5456 rows x 5 columns]

### Filter out event dates prior to threshold date in polygons with events

In [22]:
event_threshold = pd.Timestamp("01-01-2010")

In [23]:
change_metadata_df.loc[:, "change_start"] = pd.to_datetime(change_metadata_df["change_start"], dayfirst=True)

In [24]:
selected_change_polygons_indices = change_metadata_df.loc[change_metadata_df["change_start"] > event_threshold].index

In [25]:
change_metadata_df = change_metadata_df.loc[selected_change_polygons_indices]
change_signal_df = change_signal_df.loc[selected_change_polygons_indices]

In [26]:
change_signal_df

2000-01-03  2000-01-11  2000-01-18  2000-01-26  2000-01-27  \
ID  IDpix                                                               
84  172           NaN    0.419674         NaN         NaN    0.386757   
    173           NaN    0.419674         NaN         NaN    0.386757   
    174           NaN    0.413802         NaN         NaN    0.380016   
    175           NaN    0.424940         NaN         NaN    0.393733   
    176           NaN    0.436430         NaN         NaN    0.408313   
...               ...         ...         ...         ...         ...   
405 4892          NaN    0.328267    0.348205         NaN    0.323358   
    4893          NaN    0.357780    0.374025         NaN    0.319277   
    4894          NaN    0.357780    0.374025         NaN    0.319277   
    4895          NaN    0.282037    0.330697         NaN    0.312954   
    4896          NaN    0.281870    0.333567         NaN    0.283773   

           2000-01-28  2000-02-03  2000-02-11  2000-02-12  2000-02-19  ...  \
ID  IDpix                                                              ...   
84  172           NaN         NaN         NaN         NaN         NaN  ...   
    173           NaN         NaN         NaN         NaN         NaN  ...   
    174           NaN         NaN         NaN         NaN         NaN  ...   
    175           NaN         NaN         NaN         NaN         NaN  ...   
    176           NaN         NaN         NaN         NaN         NaN  ...   
...               ...         ...         ...         ...         ...  ...   
405 4892          NaN    0.355581         NaN         NaN    0.317886  ...   
    4893          NaN    0.383106         NaN         NaN    0.368284  ...   
    4894          NaN    0.383106         NaN         NaN    0.368284  ...   
    4895          NaN    0.346696         NaN         NaN    0.312491  ...   
    4896          NaN    0.316704         NaN         NaN    0.295946  ...   

           2022-12-01  2022-12-02  2022-12-08  2022-12-09  2022-12-14  \
ID  IDpix                                                               
84  172      0.323107         NaN         NaN    0.313889    0.263777   
    173      0.323107         NaN         NaN    0.313889    0.263777   
    174      0.320048         NaN         NaN    0.318409    0.259636   
    175      0.334643         NaN         NaN    0.321906    0.270980   
    176      0.317429         NaN         NaN    0.317217    0.248838   
...               ...         ...         ...         ...         ...   
405 4892     0.786440         NaN    0.835379    0.781231         NaN   
    4893     0.791635         NaN    0.793060    0.777738    0.734830   
    4894     0.791635         NaN    0.793060    0.777738    0.734830   
    4895     0.727844         NaN    0.604064    0.505488    0.500428   
    4896     0.692611         NaN    0.540881    0.487761    0.405799   

           2022-12-16  2022-12-17  2022-12-19  2022-12-24  2022-12-25  
ID  IDpix                                                              
84  172           NaN         NaN         NaN         NaN    0.301909  
    173           NaN         NaN         NaN         NaN    0.301909  
    174           NaN         NaN         NaN         NaN    0.304786  
    175           NaN         NaN         NaN         NaN    0.307882  
    176           NaN         NaN         NaN         NaN    0.302583  
...               ...         ...         ...         ...         ...  
405 4892     0.797883    0.765506         NaN    0.830963    0.776780  
    4893     0.801442    0.750359         NaN    0.786199    0.693786  
    4894     0.801442    0.750359         NaN    0.786199    0.693786  
    4895     0.483013    0.459270         NaN    0.477397    0.466940  
    4896     0.451190    0.418217         NaN    0.437109    0.398704  

[6553 rows x 2184 columns]

In [27]:
fault_detection_metadata_df = pd.concat((non_change_fault_detection_metadata_df, change_metadata_df))

In [28]:
fault_detection_signal_df = pd.concat((non_change_fault_detection_signal_df, change_signal_df))
fault_detection_signal_df = fault_detection_signal_df.reindex(sorted(fault_detection_signal_df.columns), axis=1)

In [29]:
fault_detection_metadata_df

lat        lon change_type         change_start  \
ID  IDpix                                                          
2   87    -33.058766 -71.004412      stable  2023-12-25 00:00:00   
    88    -33.059036 -71.004951      stable  2023-12-25 00:00:00   
    89    -33.058227 -71.003334      stable  2023-12-25 00:00:00   
    90    -33.058497 -71.003065      stable  2023-12-25 00:00:00   
    91    -33.058497 -71.003873      stable  2023-12-25 00:00:00   
...              ...        ...         ...                  ...   
405 4892  -34.077456 -71.633143        fire  2018-03-24 00:00:00   
    4893  -34.078264 -71.632334        fire  2018-03-24 00:00:00   
    4894  -34.076917 -71.633682        fire  2018-03-24 00:00:00   
    4895  -34.077995 -71.632873        fire  2018-03-24 00:00:00   
    4896  -34.076647 -71.633951        fire  2018-03-24 00:00:00   

          vegetation_type  
ID  IDpix                  
2   87             native  
    88             native  
    89             native  
    90             native  
    91             native  
...                   ...  
405 4892           native  
    4893           native  
    4894           native  
    4895           native  
    4896           native  

[12009 rows x 5 columns]

In [30]:
fault_detection_signal_df

2000-01-03  2000-01-04  2000-01-11  2000-01-18  2000-01-20  \
ID  IDpix                                                               
2   87       0.733178         NaN    0.804631         NaN         NaN   
    88       0.748210         NaN    0.826657         NaN         NaN   
    89       0.720108         NaN    0.791214         NaN         NaN   
    90       0.663568         NaN    0.700002         NaN         NaN   
    91            NaN         NaN    0.488417    0.388599         NaN   
...               ...         ...         ...         ...         ...   
405 4892          NaN         NaN    0.328267    0.348205         NaN   
    4893          NaN         NaN    0.357780    0.374025         NaN   
    4894          NaN         NaN    0.357780    0.374025         NaN   
    4895          NaN         NaN    0.282037    0.330697         NaN   
    4896          NaN         NaN    0.281870    0.333567         NaN   

           2000-01-26  2000-01-27  2000-01-28  2000-02-03  2000-02-05  ...  \
ID  IDpix                                                              ...   
2   87            NaN    0.794510         NaN         NaN         NaN  ...   
    88            NaN    0.796656         NaN         NaN         NaN  ...   
    89            NaN    0.776143         NaN         NaN         NaN  ...   
    90            NaN    0.704723         NaN         NaN         NaN  ...   
    91       0.374015    0.488722         NaN    0.348919         NaN  ...   
...               ...         ...         ...         ...         ...  ...   
405 4892          NaN    0.323358         NaN    0.355581         NaN  ...   
    4893          NaN    0.319277         NaN    0.383106         NaN  ...   
    4894          NaN    0.319277         NaN    0.383106         NaN  ...   
    4895          NaN    0.312954         NaN    0.346696         NaN  ...   
    4896          NaN    0.283773         NaN    0.316704         NaN  ...   

           2022-12-08  2022-12-09  2022-12-10  2022-12-14  2022-12-16  \
ID  IDpix                                                               
2   87            NaN    0.779037         NaN    0.746279         NaN   
    88            NaN    0.823661         NaN    0.787691         NaN   
    89            NaN    0.796419         NaN    0.788461         NaN   
    90            NaN    0.712442         NaN    0.715461         NaN   
    91       0.859983    0.832546         NaN         NaN         NaN   
...               ...         ...         ...         ...         ...   
405 4892     0.835379    0.781231         NaN         NaN    0.797883   
    4893     0.793060    0.777738         NaN    0.734830    0.801442   
    4894     0.793060    0.777738         NaN    0.734830    0.801442   
    4895     0.604064    0.505488         NaN    0.500428    0.483013   
    4896     0.540881    0.487761         NaN    0.405799    0.451190   

           2022-12-17  2022-12-18  2022-12-19  2022-12-24  2022-12-25  
ID  IDpix                                                              
2   87       0.736044         NaN         NaN         NaN    0.754526  
    88       0.781643         NaN         NaN         NaN    0.794315  
    89       0.778635         NaN         NaN         NaN    0.780177  
    90       0.684809         NaN         NaN         NaN    0.686562  
    91            NaN         NaN     0.79907    0.853928         NaN  
...               ...         ...         ...         ...         ...  
405 4892     0.765506         NaN         NaN    0.830963    0.776780  
    4893     0.750359         NaN         NaN    0.786199    0.693786  
    4894     0.750359         NaN         NaN    0.786199    0.693786  
    4895     0.459270         NaN         NaN    0.477397    0.466940  
    4896     0.418217         NaN         NaN    0.437109    0.398704  

[12009 rows x 2538 columns]